In [ ]:
import pandas as pd

from estival.wrappers.nevergrad import optimize_model

from tb_incubator.plotting import plot_model_vs_actual, display_plot
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm
from jax import numpy as jnp
from math import log, exp
import numpy as np
import pandas as pd
from summer2.functions.time import get_linear_interpolation_function

pd.options.plotting.backend = "plotly"

In [ ]:
def tanh_based_scaleup(t, shape, inflection_time, start_asymptote, end_asymptote=1.0):
    """
    return the function t: (1 - sigma) / 2 * tanh(b * (a - c)) + (1 + sigma) / 2
    :param shape: shape parameter
    :param inflection_time: inflection point
    :param start_asymptote: lowest asymptotic value
    :param end_asymptote: highest asymptotic value
    :return: a function
    """
    rng = end_asymptote - start_asymptote
    return (jnp.tanh(shape * (t - inflection_time)) / 2.0 + 0.5) * rng + start_asymptote

In [ ]:
targets = load_targets()

In [ ]:
def scaled_tanh(t, shape, inflection_time, min_value, max_value):
    # Base sigmoid between 0-1
    base_value = (jnp.tanh(shape * (t - inflection_time)) / 2.0 + 0.5)
    # Scale to your data range
    return min_value + base_value * (max_value - min_value)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import jax.numpy as jnp  # If you're using JAX, otherwise use regular numpy

# Define the scaled tanh function
def scaled_tanh(t, shape, inflection_time, min_value, max_value):
    base_value = (np.tanh(shape * (t - inflection_time)) / 2.0 + 0.5)
    return min_value + base_value * (max_value - min_value)

# Create an array of years (x-axis)
years = np.arange(1970, 2023, 1.0)  # From 1990 to 2022 with 0.1 intervals for smooth curve

# Example parameters - you'll adjust these to fit your data
shape = 0.10  # Adjust for steepness
inflection_time = 2008.0  # The year where the curve changes most rapidly
min_value = 25235.0  # Your minimum historical value
max_value = 804836.0  # Your maximum historical value

shape2 = 0.10  # Adjust for steepness
inflection_time2 = 2002.0  # The year where the curve changes most rapidly

# Calculate scaled tanh values
curve_values = scaled_tanh(years, shape, inflection_time, min_value, max_value)
curve_values2 = scaled_tanh(years, shape2, inflection_time2, min_value, max_value)


In [ ]:
# Create figure
fig = go.Figure()

# Add the scaled tanh curve
fig.add_trace(go.Scatter(
    x=years, 
    y=curve_values,
    mode='lines',
    name=f'Modelled detection scale up function, shape: {shape}',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=years, 
    y=curve_values2,
    mode='lines',
    name=f'Modelled detection scale up function, shape: {shape2}',
    line=dict(color='green', width=2)
))

# Add horizontal lines for asymptotes
fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[min_value, min_value],
    mode='lines',
    name='Lower Asymptote',
    line=dict(color='red', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[max_value, max_value],
    mode='lines',
    name='Upper Asymptote',
    line=dict(color='green', width=1, dash='dash')
))

# Add your historical data points (replace with your actual data)
fig.add_trace(go.Scatter(
     x=targets["notification"].index,
     y=targets["notification"],
     mode='markers',
     name='Historical Data',
     marker=dict(color='red', size=10)
))

# Update layout
fig.update_layout(
    title='Fitted Sigmoidal Curve to Historical Case Detection Data',
    xaxis_title='Year',
    yaxis_title='Notification',
    yaxis=dict(
        range=[min_value - 0.05 * (max_value - min_value), 
               max_value + 0.05 * (max_value - min_value)]
    ),
    legend=dict(x=0.02, y=0.98),
    template='plotly_white'
)

# Add a vertical line at the inflection point
fig.add_shape(
    type='line',
    x0=inflection_time, x1=inflection_time,
    y0=min_value, y1=max_value,
    line=dict(color='orange', width=1, dash='dot'),
)
fig.add_annotation(
    x=inflection_time,
    y=min_value + (max_value - min_value)/2,
    text=f'Inflection Point: {inflection_time}',
    showarrow=True,
    arrowhead=1,
    ax=-40,
    ay=0
)

# Add a vertical line at the inflection point
fig.add_shape(
    type='line',
    x0=inflection_time2, x1=inflection_time2,
    y0=min_value, y1=max_value,
    line=dict(color='orange', width=1, dash='dot'),
)
fig.add_annotation(
    x=inflection_time2,
    y=min_value + (max_value - min_value)/2,
    text=f'Inflection Point: {inflection_time2}',
    showarrow=True,
    arrowhead=1,
    ax=-40,
    ay=0
)

# Show the plot
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define the scaled tanh function
def scaled_tanh(t, shape, inflection_time, min_value, max_value):
    base_value = (np.tanh(shape * (t - inflection_time)) / 2.0 + 0.5)
    return min_value + base_value * (max_value - min_value)

# Create an array of years (x-axis)
years = np.arange(1970, 2025, 1.0)

# Fixed parameters
shape = 0.05
min_value = 25235.0
max_value = 804836.0 * 1.5

# Create a range of inflection times to compare
inflection_times = [2010.0, 2012.0, 2014.0, 2016.0, 2018.0]  # You can adjust this list

# Create figure
fig = go.Figure()

# Add curves for each inflection time with different colors
colors = ['blue', 'green', 'purple', 'orange', 'brown', 'cyan', 'magenta', 'lime']

for i, inflection_time in enumerate(inflection_times):
    # Calculate values for this inflection time
    curve_values = scaled_tanh(years, shape, inflection_time, min_value, max_value)
    
    # Add trace with a unique color from our list
    color = colors[i % len(colors)]
    
    fig.add_trace(go.Scatter(
        x=years, 
        y=curve_values,
        mode='lines',
        name=f'Inflection: {inflection_time}',
        line=dict(color=color, width=2)
    ))
    
    # Add a vertical line at each inflection point
    fig.add_shape(
        type='line',
        x0=inflection_time, x1=inflection_time,
        y0=min_value, y1=max_value,
        line=dict(color=color, width=1, dash='dot'),
    )

# Add your historical data points (replace with your actual data)
fig.add_trace(go.Scatter(
     x=targets["notification"].index,
     y=targets["notification"],
     mode='markers',
     name='Historical Data',
     marker=dict(color='red', size=10)
))

# Add horizontal lines for asymptotes
fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[min_value, min_value],
    mode='lines',
    name='Lower Asymptote',
    line=dict(color='red', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[max_value, max_value],
    mode='lines',
    name='Upper Asymptote',
    line=dict(color='red', width=1, dash='dash')
))

# Add your historical data points - uncomment when you have the data
# fig.add_trace(go.Scatter(
#     x=targets["notification"].index,
#     y=targets["notification"],
#     mode='markers',
#     name='Historical Data',
#     marker=dict(color='red', size=10)
# ))

# Update layout
fig.update_layout(
    title=f'Sigmoidal Curves with Different Inflection Times (Shape={shape})',
    xaxis_title='Year',
    yaxis_title='Notification',
    yaxis=dict(
        range=[min_value - 0.05 * (max_value - min_value), 
               max_value + 0.05 * (max_value - min_value)]
    ),
    legend=dict(x=0.02, y=0.98),
    template='plotly_white'
)

# Show the plot
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Define the scaled tanh function
def scaled_tanh(t, shape, inflection_time, min_value, max_value):
    base_value = (np.tanh(shape * (t - inflection_time)) / 2.0 + 0.5)
    return min_value + base_value * (max_value - min_value)

# Create an array of years (x-axis)
years = np.arange(1970, 2023, 1.0)

# Fixed parameters
inflection_time = 2010.0
min_value = 25235.0
max_value = 804836.0 * 1.5

# Create a range of inflection times to compare
shapes = [0.01, 0.03, 0.05, 0.07, 0.1]  # You can adjust this list

# Create figure
fig = go.Figure()

# Add curves for each inflection time with different colors
colors = ['blue', 'green', 'purple', 'orange', 'brown', 'cyan', 'magenta', 'lime']

for s, shape in enumerate(shapes):
    # Calculate values for this inflection time
    curve_values = scaled_tanh(years, shape, inflection_time, min_value, max_value)
    
    # Add trace with a unique color from our list
    color = colors[s % len(colors)]
    
    fig.add_trace(go.Scatter(
        x=years, 
        y=curve_values,
        mode='lines',
        name=f'Scale up: {shape}',
        line=dict(color=color, width=2)
    ))
    
    # Add a vertical line at each inflection point
    fig.add_shape(
        type='line',
        x0=inflection_time, x1=inflection_time,
        y0=min_value, y1=max_value,
        line=dict(color=color, width=1, dash='dot'),
    )

# Add your historical data points (replace with your actual data)
fig.add_trace(go.Scatter(
     x=targets["notification"].index,
     y=targets["notification"],
     mode='markers',
     name='Historical Data',
     marker=dict(color='red', size=10)
))

# Add horizontal lines for asymptotes
fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[min_value, min_value],
    mode='lines',
    name='Lower Asymptote',
    line=dict(color='red', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=[years.min(), years.max()],
    y=[max_value, max_value],
    mode='lines',
    name='Upper Asymptote',
    line=dict(color='red', width=1, dash='dash')
))

# Add your historical data points - uncomment when you have the data
# fig.add_trace(go.Scatter(
#     x=targets["notification"].index,
#     y=targets["notification"],
#     mode='markers',
#     name='Historical Data',
#     marker=dict(color='red', size=10)
# ))

# Update layout
fig.update_layout(
    title=f'Sigmoidal Curves with Different Scale Up Shapes (Inflection time={inflection_time})',
    xaxis_title='Year',
    yaxis_title='Notification',
    yaxis=dict(
        range=[min_value - 0.05 * (max_value - min_value), 
               max_value + 0.05 * (max_value - min_value)]
    ),
    legend=dict(x=0.02, y=0.98),
    template='plotly_white'
)

# Show the plot
fig.show()